Primero, creamos el script que nos permitirá conectarnos a nuestra base de datos en SQL

In [ ]:
import pandas as pd #librería para gestionar dataframes
import pyodbc #librería que sirve para conectarse a diferentes tipos de bases de datos
from sqlalchemy import create_engine #librería encargada de crear el engine

# Configuración de la conexión (en este caso usa los parámetros del propio windows)
server = "LAPTOP-Arturo" #especificar el nombre del servidor con el que te vas a conectar
database = "Acciones" #el nombre de la base de datos con la que vas a trabajar

# Conectar la base de datos
conn_str = f"mssql+pyodbc://@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server" 
engine = create_engine(conn_str)

Acabamos de generar una variable denominada "engine" que es nuestra "llave" para conectarnos con la base de datos SQL server.
Cabe destacar que esta es una versión simplificada que no solicita ni usuario ni contraseña, usa los parámetros de inicio de sesión de Windows

Ahora, vamos paso a paso a subir un dataframe a SQL, primero debemos importar uno, usaremos los datos históricos de acciones de una empresa:

In [ ]:
#importamos la librería para acceder a los datos de Yahoo Finance
import yfinance as yf 

Ticker = 'NVDA'  #este "Ticker" es el código con el que se identifica a la empresa
Nvidia1 = yf.download(Ticker, start='2020-01-01', end='2024-12-31')

Nvidia1.head(10) #nos mostrará las primeras filas

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,NVDA,NVDA,NVDA,NVDA,NVDA
Date,,,,,
2020-01-02,5.972162,5.972162,5.892753,5.943286,237536000
2020-01-03,5.876571,5.920383,5.827531,5.852424,205384000
2020-01-06,5.901215,5.906442,5.757083,5.783220,262636000
2020-01-07,5.972659,6.018463,5.884537,5.929594,314856000
2020-01-08,5.983861,6.025184,5.928349,5.968427,277108000
2020-01-09,6.049580,6.122020,5.995811,6.070242,255112000
2020-01-10,6.081941,6.187240,6.067752,6.156870,316296000
2020-01-13,6.272624,6.297767,6.142432,6.165085,319840000


Vemos que las columnas que se generaron son de más de un nivel, esto podría generar problemas para importarlo a SQL, antes de eso debemos eliminar el segundo nivel y asegurarnos que se importe correctamente

In [ ]:
# eliminamos el segundo nivel (el que tiene el nombre de las acciones)
Nvidia1.columns = Nvidia1.columns.droplevel(1) 

#verificamos que se haya eliminado
Nvidia1.head(10)

Price,Close,High,Low,Open,Volume
Date,,,,,
2020-01-02,5.972162,5.972162,5.892753,5.943286,237536000
2020-01-03,5.876571,5.920383,5.827531,5.852424,205384000
2020-01-06,5.901215,5.906442,5.757083,5.783220,262636000
2020-01-07,5.972659,6.018463,5.884537,5.929594,314856000
2020-01-08,5.983861,6.025184,5.928349,5.968427,277108000
2020-01-09,6.049580,6.122020,5.995811,6.070242,255112000
2020-01-10,6.081941,6.187240,6.067752,6.156870,316296000
2020-01-13,6.272624,6.297767,6.142432,6.165085,319840000
2020-01-14,6.155626,6.255199,6.142432,6.229808,359088000


In [18]:
Nvidia1.to_sql('Nvidia', engine, if_exists='replace', index=True)

210

Esta última función lo que hace en términos generales es importar el dataframe llamado "Nvidia1" a una tabla en SQL server llamada "Nvidia", 
en caso esta tabla ya exista se reemplazará con la data de este nuevo dataframe, el parámetro de "index" está en "True" para que se importe 
la fecha, que en el dataframe la reconoce como el índice (la columna 0)

Ahora probemos si resultó, usemos el "engine" para realizar una consulta a la base de datos, tal como haríamos si estuvieramos en 
la interfaz de SSMS (SQL Server Management Studio)

In [33]:
Nvidia2023 = pd.read_sql("""
                        select * from Nvidia
                        where YEAR(Date)=2023
                        order by Date ASC""", engine)

# Mostrar las primeras filas
Nvidia2023.head(10)

,Date,Close,High,Low,Open,Volume
0,2023-01-03,14.303279,14.983722,14.084458,14.838840,401277000
1,2023-01-04,14.736925,14.840840,14.229342,14.555075,431324000
2,2023-01-05,14.253321,14.552076,14.136416,14.479135,389168000
3,2023-01-06,14.846834,14.997711,14.022510,14.462149,405044000
4,2023-01-09,15.615206,16.042855,15.128604,15.271488,504231000
5,2023-01-10,15.895974,15.948930,15.459332,15.494303,384101000
6,2023-01-11,15.987899,16.014877,15.550258,15.827031,353285000
7,2023-01-12,16.497480,16.623377,15.479315,16.086818,551409000
8,2023-01-13,16.885164,16.908146,16.151766,16.264672,447287000
9,2023-01-17,17.687506,17.713485,16.885164,16.885164,511102000


La función read_sql que pertenece a Pandas, permite redactar una consulta como si se tratara de SQL server, esto nos permite extraer datos usando 
consultas desde las mas sencillas hasta las mas complejas, usando como "llave", el "engine" que habíamos usado antes.

Por último, con el objetivo de demostrar que efectivamente la consulta funcionó y se extrajo los datos de las acciones únicamente para el año 2023

In [ ]:
#extrae el primer dato que se identifica (fila número 0)
primer_fila = Nvidia2023.head(1) 

#extrae el último dato del que se tiene registro (fila número 249)
ultima_fila = Nvidia2023.tail(1) 

df_unido = pd.concat([primer_fila, ultima_fila], axis=0)
df_unido

,Date,Close,High,Low,Open,Volume
0,2023-01-03,14.303279,14.983722,14.084458,14.838840,401277000
249,2023-12-29,49.503410,49.978234,48.732700,49.794301,389293000
